In [ ]:
%pylab inline

In many scenarios, either in simulations or when working with real-world data, it may be necessary to use concepts from geometry when programming. In the following, we are going to have a look at some possible scenarios, and in the second hal of the notebook, we are going to add geometries to maps created from several input formats.

# Points, polygons and lines

Perhaps the most common Python library for dealing with shapes is called [`shapely`](https://shapely.readthedocs.io/en/stable/manual.html)

### Spatial Data Model

The fundamental types of geometric objects implemented by Shapely are *points*, *curves*, and *surfaces*. Each is associated with three sets of points in the plane. The *interior*, *boundary*, and *exterior* sets of a feature are mutually exclusive.

* A *Point* has an *interior* set of exactly one point, a *boundary* set of exactly no points, and an *exterior* set of all other points. A Point has a topological dimension of 0.
* A *Curve* has an *interior* set consisting of the infinitely many points along its length (imagine a *Point* dragged in space), a *boundary* set consisting of its two end points, and an *exterior* set of all other points. A *Curve* has a topological dimension of 1.
* A *Surface* has an *interior* set consisting of the infinitely many points within (imagine a *Curve* dragged in space to  over an area), a *boundary* set consisting of one or more *Curves*, and an *exterior* set of all other points including those within holes that might exist in the surface. A *Surface* has a topological dimension of 2.

Let's import them!

In [ ]:
from shapely.geometry import Point,Polygon,LineString

Let us define a point! We create an object of the `Point()` class, that can be in 2 or 3 dimensions.

In [ ]:
p = Point([0,0])

In [ ]:
p

or a line

In [ ]:
l = LineString([[-2,1],[3,5]])

In [ ]:
l

We can create polygons by giving a list of coordinates to the constructor of the `Polygon()` class. We don't have to duplicate the first coordinate in the last element, but then our polygon won't be closed.

In [ ]:
square = Polygon([[-1,-1],[-1,1],[1,1],[1,-1]])

In a Jupyter notebook, we can easily display our shapes.

In [ ]:
square

Let's write a function that creates regular polygons!

In [ ]:
import numpy as np

In [ ]:
def regular(n,R=1.5):
    t = np.linspace(0,2*np.pi,n+1)
    return Polygon([[R*np.cos(elem),R*np.sin(elem)] for elem in t])

In [ ]:
fivefold = regular(5)
fivefold

Let's make some simple operations with our polygons:

In [ ]:
fivefold.area

In [ ]:
fivefold.centroid

In [ ]:
fivefold.union(square)

In [ ]:
fivefold.difference(square)

In [ ]:
fivefold.buffer(0.4)

In [ ]:
square.buffer(1)

In [ ]:
fivefold.boundary

In [ ]:
fivefold.boundary.geom_type

# Simple maps


If we do the data analysis in Python, then it would be convenient to put our data on a map in Python, too. There are some libraries that enable the usage of the powerful and lightweight [Leaflet.js](https://leafletjs.com/) library in Python. In the following cells, we are going to have a look at some basic functionalities of these maps in Folium. You may use another library as you choose.

In [ ]:
import folium

Let's display the inner city of Budapest in a Jupyter Notebook!

In [ ]:
folium.Map(location=[47.5,19.05],zoom_start=13)

The map used the tiles of the openstreetmap.org freely available database. There are many other options depending on our aims, see https://deparkes.co.uk/2016/06/10/folium-map-tiles/. A rather artistic approach for example is:

In [ ]:
folium.Map(location=[47.5,19.05],zoom_start=13,tiles='stamenwatercolor')

Let's stick to a more scientific version, and let's add some useful objects to our map!

In [ ]:
mymap = folium.Map(location=[47.5,19.05],zoom_start=13,tiles='cartodbpositron')
marker = folium.Marker(location=[47.47,19.06],popup='ELTE')
marker.add_to(mymap)
mymap

In [ ]:
mymap = folium.Map(location=[47.5,19.05],zoom_start=13,tiles='cartodbpositron')
marker = folium.Marker(location=[47.47,19.06],popup='ELTE')
mymap.add_child(marker)

Which coordinate reference system does our map use? https://epsg.io/3857

In [ ]:
mymap.crs

In [ ]:
mymap.get_bounds()

In [ ]:
mymap.fit_bounds(mymap.get_bounds())
mymap

## Geopandas

The ultimate tool to read almost any kind of geographical data is GeoPandas.

The goal of GeoPandas is to make working with geospatial data in python easier. It combines the capabilities of pandas and shapely, providing geospatial operations in pandas and a high-level interface to multiple geometries to shapely. GeoPandas enables you to easily do operations in python that would otherwise require a spatial database such as PostGIS.

In [ ]:
import geopandas as gpd

We are going to try out some simple operations on Hungarian county and highway data. You can download the county dataset from here: https://data2.openstreetmap.hu/hatarok/index.php?admin=6. It contains so-called shapefiles that store the geometric information and the metadata as well. The `.shp` file can be opened in geopandas as follows:

In [ ]:
megyek = gpd.read_file('shapes/admin6.shp', encoding = 'utf8')
megyek.head()

The geometry column contains the already known shapely `Polygon` class objects! Let's check Borsod-Abaúj-Zemplén:

In [ ]:
baz = megyek['geometry'][0]

Which coordinate system did this shapefile use? http://geopandas.org/projections.html

In [ ]:
megyek.crs

Which county has the biggest area?

In [ ]:
megyek['NAME'][megyek.area.idxmax]

Let us create a list of neighboring counties.

First, we 'inflate' our polygons.

In [ ]:
from copy import deepcopy

In [ ]:
big_megyek = deepcopy(megyek)
big_megyek['geometry'] = big_megyek['geometry'].buffer(10)
big_megyek2 = deepcopy(big_megyek)

In [ ]:
big_megyek2

In [ ]:
neighbors = gpd.sjoin(big_megyek,big_megyek2,how ='left',op='intersects')[['NAME_left','NAME_right']]

In [ ]:
neighbors.head(9)

## Reprojecting and quick plotting geopandas data

In [ ]:
railways = gpd.read_file('shapes/gis.osm_railways_free_1.shp')
railways.plot()

In [ ]:
railways.crs

In [ ]:
railways.to_crs({'init': 'epsg:3395'}, inplace=True)
railways.plot()

## How long railways do Hungarian counties have?

We should reproject the county dataset to match the projection of the railway dataset.

In [ ]:
# That would be the command without an R-Tree search. This takes too long!
# rr = railways.intersects(geometry_BAZ_megye)

In [ ]:
megyek.to_crs({'init': 'epsg:3395'}, inplace=True)

Sometimes it is useful to speed up spatial operations using a spatial index. Here is a good introduction into R-trees in geopandas.

http://geoffboeing.com/2016/10/r-tree-spatial-index-python/

Creating an R-tree index:

In [ ]:
spatial_index = railways.sindex

In [ ]:
type(spatial_index)

In [ ]:
megyek.head()

In [ ]:
spatial_index = railways.sindex

#The intersection of the railway system with each of the counties
megyek['possible_matches'] = megyek['geometry'].map(lambda g: list(spatial_index.intersection(g.bounds)))

Trying out the concept.

Take a look at the new column in *megyek*:

In [ ]:
megyek

In [ ]:
railways

In [ ]:
railways.iloc[railway_BAZ_megye].plot()

In [ ]:
def filter_railways(indices, polygon):
    #Selects points 
    sdf = railways.iloc[indices]
    return list(sdf[sdf.intersects(polygon)].index)

In [ ]:
railway_BAZ_megye = megyek['possible_matches'][0] #The intersection of the railway system with the given county
geometry_BAZ_megye = megyek['geometry'][0]   
railways.loc[filter_railways(railway_BAZ_megye, geometry_BAZ_megye)].plot()

Creating the intersections, calculating the lengths in km:

In [ ]:
megyek['matches'] = megyek.apply(lambda row: filter_railways(row['possible_matches'], row['geometry']),axis=1)
megyek['lengths']= megyek['matches'].map(lambda m: sum(railways.loc[m].length)/1000)

megyek.sort_values(by='lengths',ascending=False)

## Adding shapes to a folium map - Railtracks in Budapest

In [2]:
import json, folium

We use the same basemap as before, and we retrieve the railways of Budapest from our previous analysis. Then, we reproject the railways to use *lon, lat* coordinates again, and convert them to GeoJSON, which is then added by folium to the map.

What is JSON: https://en.wikipedia.org/wiki/JSON

Tutorial with JSONs: https://www.w3schools.com/jS/js_json_intro.asp

In [ ]:
railways.head()

In [ ]:
mymap = folium.Map(location=[47.5,19.05], zoom_start=13, tiles='cartodbpositron')
budapest_railways = railways.loc[megyek['matches'][12]].to_crs({'init':'epsg:4326'})
bpr_geojson = json.loads(budapest_railways.to_json())

In [ ]:
budapest_railways.head()

In [ ]:
bpr_geojson

Displaying the results.

In [ ]:
folium.GeoJson(bpr_geojson).add_to(mymap)
mymap